In [1]:
from utils import *
import pandas as pd
import mne
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.preprocessing import normalize, StandardScaler
plt.style.use('seaborn-whitegrid')

TYPE_DEF = {0:'Non-Stress', 1:'Neutral', 2: 'Stress'}
PSS = load('PSS')
type_count = load('type_count')

In [2]:
def get_freq(PSS):
    # peak at info
    temp = PSS.popitem()
    PSS[temp[0]] = temp[1]
    raw = temp[1]['raw']
    power,freq = mne.time_frequency.psd_welch(raw,n_fft=125, verbose=True)
    return freq


for name, info in PSS.items():
    raw = info['raw']
    raw.filter(l_freq=1,h_freq=None, method='iir', iir_params={'order':3.0, 'ftype':'butter'}, verbose=False) # Slow drift
    raw.notch_filter(freqs=[50])

freq = get_freq(PSS)
print(freq)


band_names = np.array(['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma', 'Slow', 'Low_beta'])
filter_list = [[1,3],[4,7],[8,12],[13,30],[30,43], [4,13], [13,17]]
bands = []
for filt in filter_list:
    pt = np.argwhere((freq >= filt[0]) & (freq <= filt[1])).reshape(-1)
    bands.append(pt)
bands = np.array(bands)
print(bands)

def get_markers():
    sampling_rate = 125 #Hz
    # 15/60 = 0.25
    step_minutes = np.arange(0,5,0.25)
    print(f"{step_minutes=}")
    step_minutes = np.expand_dims(step_minutes * sampling_rate * 60, axis=1)
    markers = np.concatenate( [step_minutes, np.zeros( step_minutes.shape ), np.ones( step_minutes.shape ) ], axis=1  ).astype(np.int64)
    return markers
markers = get_markers()
# markers

Effective window size : 1.000 (s)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62.]
[array([1, 2, 3]) array([4, 5, 6, 7]) array([ 8,  9, 10, 11, 12])
 array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
        30])
 array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43])
 array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13])
 array([13, 14, 15, 16, 17])]
step_minutes=array([0.  , 0.25, 0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 ,
       2.75, 3.  , 3.25, 3.5 , 3.75, 4.  , 4.25, 4.5 , 4.75])


/tmp/ipykernel_3668/3442835183.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bands = np.array(bands)


In [3]:
# features = None
for name,info in PSS.items():
    raw = info['raw']
    epochs = mne.Epochs(raw, markers, tmin=0, tmax=15, baseline=(0,15), verbose=False)
    feature_epochs = None
    for evoked in epochs.iter_evoked():
        feature = None
        slow, gamma = None, None
        a_f3, a_f4 = None, None
        a_t7, a_t8 = None, None
        b_f3, b_f4 = None, None
        b_t7, b_t8 = None, None
        for index, band in enumerate(bands):
            power,freq = mne.time_frequency.psd_welch(evoked,n_fft=125, verbose=False)
            power = power.squeeze()
            power = 10 * np.log10(power)
            data = power[::,band].mean(axis=1).reshape(1,-1)
            # for asym
            if(band_names[index] == 'Alpha'):
                a_f3 = data[:,raw.ch_names.index('F3')]
                a_f4 = data[:,raw.ch_names.index('F4')]
                # We use t3 as t7 and t4 as t8
                a_t7 = data[:,raw.ch_names.index('T3')]
                a_t8 = data[:,raw.ch_names.index('T4')]
            if(band_names[index] == 'Beta'):
                b_f3 = data[:,raw.ch_names.index('F3')]
                b_f4 = data[:,raw.ch_names.index('F4')]
                # We use t3 as t7 and t4 as t8
                b_t7 = data[:,raw.ch_names.index('T3')]
                b_t8 = data[:,raw.ch_names.index('T4')]

            ####### Mean for visualization #######
            data = data.mean().reshape(1,-1)
            # for relative gamma
            if(band_names[index] == 'Slow'): slow = data
            if(band_names[index] == 'Gamma'): gamma = data

            if(type(feature) == type(None)): feature = data
            else: feature = np.concatenate([feature, data], axis=1)

        # the eighth feature: relative gamma is slow/gamma
        relative_gamma = slow/gamma
        feature = np.concatenate([feature, relative_gamma], axis=1)
        # The asymetry
        alpha_frontal = ((a_f4 - a_f3) / (a_f4 + a_f3)).reshape(1,-1)
        feature = np.concatenate([feature, alpha_frontal], axis=1)
        # alpha_temporal
        alpha_temporal = ((a_t8 - a_t7) / (a_t8 + a_t7)).reshape(1,-1)
        feature = np.concatenate([feature, alpha_temporal], axis=1)
        # alpha_asymmetry
        alpha_asymmetry = alpha_frontal + alpha_temporal
        feature = np.concatenate([feature, alpha_asymmetry], axis=1)
        # beta_frontal
        beta_frontal = ((b_f4 - b_f3) / (b_f4 + b_f3)).reshape(1,-1)
        feature = np.concatenate([feature, beta_frontal], axis=1)
        # beta_temporal
        beta_temporal = ((b_t8 - b_t7) / (b_t8 + b_t7)).reshape(1,-1)
        feature = np.concatenate([feature, beta_temporal], axis=1)

        if(type(feature_epochs) == type(None)): feature_epochs = feature
        else: feature_epochs = np.concatenate( [feature_epochs, feature], axis=0 )
    info['feature'] = feature_epochs
print(f"{feature_epochs.shape=}")


feature_epochs.shape=(20, 13)


In [10]:
# plt.plot(feature)
feature_names = list(band_names)
feature_names.append('Relative_Gamma')
feature_names.append('Alpha_Frontal')
feature_names.append('Alpha_Temporal')
feature_names.append('Alpha_Asymmetry')
feature_names.append('Beta_Frontal')
feature_names.append('Beta_Temporal')
feature_names = np.array(feature_names)
feature_names[[3,10]]
X_ori,y_ori = [], []
filtered_participants = []
filtered_scored = []
names = []
non_stress_count, stress_count, neutral = 0,0,0
for index,(name,info) in enumerate(PSS.items()):
    names.append(name)
    for feature in info['feature']:
        # Neutral
        if(info['type'] == 1):
            neutral = neutral + 1
            y_ori.append(2)
        # if(info['type'] == 1 or (info['score'] >= 17 and info['score'] <= 23)): continue
        # Non-Stress
        elif(info['type'] == 0):
            non_stress_count = non_stress_count + 1
            # print(name, info['score'])
            y_ori.append(0)
        # Stress
        elif(info['type'] == 2):
            stress_count = stress_count + 1
            y_ori.append(1)
        X_ori.append(feature)
        filtered_participants.append(name)
        filtered_scored.append(info['score'])
print(f"{non_stress_count=} {stress_count=}")
print(f"{np.array(X_ori).shape=}")
print(f"{np.array(y_ori).shape=}")
# save(X_ori, "X_ori_cut17-23")
# save(y_ori, "y_ori_cut17-23")

non_stress_count=320 stress_count=400
np.array(X_ori).shape=(1100, 13)
np.array(y_ori).shape=(1100,)


In [6]:
np.savetxt("foo.csv", np.array(X_ori), delimiter=",")

In [8]:
np.savetxt("y.csv", np.array(y_ori), delimiter=",")

In [11]:
names

['amp',
 'aui',
 'aun',
 'bam',
 'bank',
 'bas',
 'beau',
 'beer',
 'boss',
 'cedric',
 'dream',
 'dt',
 'eiyu',
 'fabby',
 'fahmai',
 'film',
 'flm',
 'gon',
 'harold',
 'ice',
 'int',
 'job',
 'kant',
 'kao',
 'kee',
 'miiw',
 'mind',
 'minkhant',
 'MJ',
 'mu',
 'new',
 'nice',
 'nisit',
 'noey',
 'nopphon',
 'nuclear',
 'pang',
 'pear',
 'pla',
 'ploy',
 'poon',
 'por',
 'praewphan',
 'prin',
 's',
 'sam',
 'satya',
 'shin',
 'suyo',
 'taew',
 'tom',
 'tor',
 'yee',
 'yong',
 'younten']

In [14]:
feature_names

array(['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma', 'Slow', 'Low_beta',
       'Relative_Gamma', 'Alpha_Frontal', 'Alpha_Temporal',
       'Alpha_Asymmetry', 'Beta_Frontal', 'Beta_Temporal'], dtype='<U15')

In [12]:
pd.read_csv('foo.csv')

,Name,type,Delta,Theta,Alpha,Beta,Gamma,Slow,Low_beta,Relative_Gamma,Alpha_Frontal,Alpha_Temporal,Alpha_Asymmetry,Beta_Frontal,Beta_Temporal
0,amp,0,-111.529686,-115.696422,-113.324383,-122.948584,-125.431633,-114.888972,-120.079879,0.915949,0.011724,0.003254,0.014978,0.017499,0.010344
1,amp,0,-110.872188,-112.369937,-108.712121,-121.175530,-125.453975,-110.886184,-117.249289,0.883879,0.006065,0.002454,0.008519,0.018412,0.006791
2,amp,0,-112.413112,-114.241355,-109.740452,-121.817560,-125.627243,-112.390630,-118.435911,0.894636,0.005862,0.005900,0.011762,0.018631,0.008374
3,amp,0,-112.817834,-114.840791,-111.092637,-122.393557,-124.689961,-113.459119,-119.145567,0.909930,0.008790,0.013047,0.021837,0.016485,0.009304
4,amp,0,-112.463699,-115.683811,-112.565514,-123.176237,-126.035360,-114.462702,-120.163493,0.908179,0.003897,0.000862,0.004759,0.016392,0.008096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,younten',2,-107.726493,-113.698609,-114.721624,-119.445003,-121.262649,-114.565328,-118.263058,0.944770,-0.006399,0.003592,-0.002807,-0.005274,0.002239
1096,younten',2,-107.526039,-113.258700,-115.764457,-120.157652,-122.231807,-115.057083,-119.318195,0.941302,-0.003508,0.008772,0.005264,-0.005222,0.003614
1097,younten',2,-106.106712,-112.484702,-115.685483,-120.320611,-122.243750,-114.669686,-118.939252,0.938041,-0.012073,0.003833,-0.008240,-0.005539,0.004592
1098,younten',2,-106.775438,-112.570303,-115.015830,-120.468343,-122.381889,-114.336985,-119.393294,0.934264,-0.010993,0.001989,-0.009005,-0.003999,0.003941
